In [0]:
#install google spread && PyDrive
!pip install --upgrade -q gspread PyDrive
!pip install pygeocoder
!pip install geopandas
!pip install descartes

    100% |████████████████████████████████| 993kB 23.3MB/s 
  Stored in directory: /root/.cache/pip/wheels/7c/4c/00/d05c66c4af5411c554c91b8079732c8a0359c2226fb8c01031
Successfully built pygeocoder
    100% |████████████████████████████████| 931kB 22.7MB/s 
    100% |████████████████████████████████| 17.9MB 1.8MB/s 
    100% |████████████████████████████████| 10.8MB 3.8MB/s 
  Stored in directory: /root/.cache/pip/wheels/db/bf/bc/06a3e1bfe0ab27d2e720ceb3cff3159398d92644c0cec2c125
Successfully built munch


In [0]:
import math
import os
import string
from datetime import datetime
import re
import json
import csv
import pickle
import requests
import collections
from pprint import pformat
from sklearn.metrics import f1_score,precision_score,recall_score
from sklearn.preprocessing.label import LabelEncoder
import tensorflow as tf
# from pygeocoder import Geocoder
from shapely.geometry import Point
import geopandas

In [0]:
# Imports
# Numpy,Pandas
import numpy as np
import pandas as pd
import datetime

# matplotlib,seaborn,pyecharts

import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
# plt.style.use('ggplot')  #风格设置近似R这种的ggplot库
import seaborn as sns
sns.set_style('whitegrid')
%matplotlib inline
import missingno as msno

# import sklearn

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import auc
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
from sklearn.metrics import recall_score
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler


#  忽略弹出的warnings
import warnings
warnings.filterwarnings('ignore')  

pd.set_option('display.float_format', lambda x: '%.4f' % x)

from imblearn.over_sampling import SMOTE
import itertools

In [0]:
#Goole Drive libs configuration
from oauth2client.client import GoogleCredentials
from google.colab import auth
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
import gspread
#@title Authenticate Google Drive client && create local directory
auth.authenticate_user()
credential = GoogleCredentials.get_application_default()
#iniate Google Spread Sheet
gc = gspread.authorize(credential)
#iniate Google Drive
gauth = GoogleAuth()
gauth.credentials = credential
drive = GoogleDrive(gauth)

# #2. Get the file
# downloaded = drive.CreateFile({'id':'1BH-rffqv_1auzO7tdubfaOwXzf278vJK'}) # replace the id with id of file you want to access
# downloaded.GetContentFile('xyz.csv') 

#Import file from Google Drive
def importfile(file_id='1W_SHJkvrYSCLXitec33iy7-MKG854ZnF'):
    f = drive.CreateFile({'id': file_id})
    print('Drive file (title: %s, id: %s)' % (f['title'], f['id']))
    fname = os.path.join(os.getcwd(), f['title'])
    print('file path: {}'.format(fname))
    f_ = drive.CreateFile({'id': f['id']})
    f_.GetContentFile(fname)
    return fname

def exportfile(file_name="sample.txt"):
    uploaded = drive.CreateFile({'title':file_name})
    uploaded.Upload()
    print('file {} uploaded to Google Drive'.format(file_name))
    
def create_dir(dir_name="Sample"):
  dir_path=os.path.join(os.getcwd(),dir_name)
  if not os.path.exists(dir_path):
    os.makedirs(dir_name)
  return dir_path


CWD=os.getcwd()
print("Current root dir: ",CWD)

Current root dir:  /content


In [0]:
Capstone = importfile(file_id = '1-1lVd9sPctCOOrpxJ8vpf7qf6zhU3EhN')
Capstone = pd.read_csv(os.path.join("/content","capstone.csv"), header = 0, index_col = 0, keep_default_na = False)
Capstone.head()

Drive file (title: capstone.csv, id: 1-1lVd9sPctCOOrpxJ8vpf7qf6zhU3EhN)
file path: /content/capstone.csv


dataid vehicle_type model_year quick_charge_port delivery_date  \
0     410  Nissan Leaf       2016             False     unknown_1   
1     410  Nissan Leaf       2016             False     unknown_1   
2     410  Nissan Leaf       2016             False     unknown_1   
3     410  Nissan Leaf       2016             False     unknown_1   
4     410  Nissan Leaf       2016             False     unknown_1   

        ownership_status      localhour       car1 any_electric_vehicle  \
0  Leased by Participant  1/29/17 11:00  unknown_1                  Yes   
1  Leased by Participant  4/20/17 13:00  unknown_1                  Yes   
2  Leased by Participant   5/7/17 12:00  unknown_1                  Yes   
3  Leased by Participant   5/7/17 13:00  unknown_1                  Yes   
4  Leased by Participant  5/10/17 11:00  unknown_1                  Yes   

     status  ...  electric_vehicle_buy_reason_envir  \
0  Complete  ...              Environmental benefit   
1  Complete  ...              Environmental benefit   
2  Complete  ...              Environmental benefit   
3  Complete  ...              Environmental benefit   
4  Complete  ...              Environmental benefit   

  electric_vehicle_buy_reason_save_money  \
0                              unknown_1   
1                              unknown_1   
2                              unknown_1   
3                              unknown_1   
4                              unknown_1   

  electric_vehicle_buy_reason_new_tech_interest  \
0                                     unknown_1   
1                                     unknown_1   
2                                     unknown_1   
3                                     unknown_1   
4                                     unknown_1   

  electric_vehicle_buy_reason_performance  \
0  Acceleration and performance of EVâ€™s   
1  Acceleration and performance of EVâ€™s   
2  Acceleration and performance of EVâ€™s   
3  Acceleration and performance of EVâ€™s   
4  Acceleration and performance of EVâ€™s   

  electric_vehicle_buy_reason_quietness  \
0                             Quietness   
1                             Quietness   
2                             Quietness   
3                             Quietness   
4                             Quietness   

  electric_vehicle_buy_reason_other_recommended electric_vehicle_buy_again  \
0                                     unknown_1                         No   
1                                     unknown_1                         No   
2                                     unknown_1                         No   
3                                     unknown_1                         No   
4                                     unknown_1                         No   

     city  state label  
0  Austin  Texas     1  
1  Austin  Texas     1  
2  Austin  Texas     1  
3  Austin  Texas     1  
4  Austin  Texas     1  

[5 rows x 41 columns]

In [0]:
Capstone.columns

Index(['dataid', 'vehicle_type', 'model_year', 'quick_charge_port',
       'delivery_date', 'ownership_status', 'localhour', 'car1',
       'any_electric_vehicle', 'status', 'spend_time_at_home_monday',
       'spend_time_at_home_tuesday', 'spend_time_at_home_wednesday',
       'spend_time_at_home_thursday', 'spend_time_at_home_friday',
       'ethnicity_asian_pacific_islander',
       'ethnicity_caucasian_other_than_hispanic_or_latino',
       'ethnicity_hispanic_or_latino', 'ethnicity_decline',
       'residents_under_5', 'residents_6_to_12', 'residents_13_to_18',
       'residents_19_to_24', 'residents_25_to_34', 'residents_35_to_49',
       'residents_50_to_64', 'residents_65_and_older', 'education_level',
       'total_annual_income', 'electric_vehicle_satisfied',
       'electric_vehicle_features_liked', 'electric_vehicle_buy_reason_envir',
       'electric_vehicle_buy_reason_save_money',
       'electric_vehicle_buy_reason_new_tech_interest',
       'electric_vehicle_buy_reason_

In [0]:
Capstone.vehicle_type.replace(['unknown_0', 'unknown_1'], 'unfilled', inplace = True)
Capstone.vehicle_type.replace(['Nissan Leaf', 'Chevrolet Volt', 'Mitsubishi i-MiEV', 'Tesla Model S'], 'filled', inplace = True)

Capstone.model_year.replace(['unknown_0', 'unknown_1'], 'unfilled', inplace = True)
Capstone.model_year.replace(['2016', '2012', '2013', '2011', 2013, 2012, '2014', '2008', 2014, 2014, 2011, 2015, '2015'], 'filled', inplace = True)

Capstone.quick_charge_port.replace(['unknown_0'], 'unfilled', inplace = True)

Capstone.ownership_status.replace(['unknown_0', 'unknown_1'], 'unfilled', inplace = True)
Capstone.ownership_status.replace(['2016', '2012', '2013', '2011', 2013, 2012, '2014', '2008', 2014, 2014, 2011, 2015, '2015'], 'filled', inplace = True)

Capstone.spend_time_at_home_monday.replace(['unknown_0', 'unknown_1'], 0, inplace = True)
Capstone.spend_time_at_home_monday.replace(['Monday'], 1, inplace = True)

Capstone.spend_time_at_home_tuesday.replace(['unknown_0', 'unknown_1'], 0, inplace = True)
Capstone.spend_time_at_home_tuesday.replace(['Tuesday'], 1, inplace = True)

Capstone.spend_time_at_home_wednesday.replace(['unknown_0', 'unknown_1'], 0, inplace = True)
Capstone.spend_time_at_home_wednesday.replace(['Wednesday'], 1, inplace = True)

Capstone.spend_time_at_home_thursday.replace(['unknown_0', 'unknown_1'], 0, inplace = True)
Capstone.spend_time_at_home_thursday.replace(['Thursday'], 1, inplace = True)

Capstone.spend_time_at_home_friday.replace(['unknown_0', 'unknown_1'], 0, inplace = True)
Capstone.spend_time_at_home_friday.replace(['Friday'], 1, inplace = True)

Capstone.ethnicity_asian_pacific_islander.replace(['unknown_0', 'unknown_1'], 0, inplace = True)
Capstone.ethnicity_asian_pacific_islander.replace(['Asian/Pacific Islander'], 1, inplace = True)

Capstone.ethnicity_caucasian_other_than_hispanic_or_latino.replace(['unknown_0', 'unknown_1'], 0, inplace = True)
Capstone.ethnicity_caucasian_other_than_hispanic_or_latino.replace(['Caucasian'], 1, inplace = True)

Capstone.ethnicity_hispanic_or_latino.replace(['unknown_0', 'unknown_1'], 0, inplace = True)
Capstone.ethnicity_hispanic_or_latino.replace(['Hispanic'], 1, inplace = True)

Capstone.ethnicity_decline.replace(['unknown_0', 'unknown_1'], 0, inplace = True)
Capstone.ethnicity_decline.replace(['Decline to Respond'], 1, inplace = True)

Capstone.residents_under_5.replace(['unknown_0', 'unknown_1'], 0, inplace = True)
Capstone.residents_6_to_12.replace(['unknown_0', 'unknown_1'], 0, inplace = True)
Capstone.residents_13_to_18.replace(['unknown_0', 'unknown_1'], 0, inplace = True)
Capstone.residents_19_to_24.replace(['unknown_0', 'unknown_1'], 0, inplace = True)
Capstone.residents_25_to_34.replace(['unknown_0', 'unknown_1'], 0, inplace = True)
Capstone.residents_35_to_49.replace(['unknown_0', 'unknown_1'], 0, inplace = True)
Capstone.residents_50_to_64.replace(['unknown_0', 'unknown_1'], 0, inplace = True)
Capstone.residents_65_and_older.replace(['unknown_0', 'unknown_1'], 0, inplace = True)

Capstone.electric_vehicle_satisfied.replace(['unknown_0', 'unknown_1'], 'unfilled', inplace = True)

Capstone.electric_vehicle_features_liked.replace(['unknown_0', 'unknown_1'], 'unfilled', inplace = True)

Capstone.electric_vehicle_buy_reason_envir.replace(['unknown_0', 'unknown_1'], 'unfilled', inplace = True)

Capstone.electric_vehicle_buy_reason_save_money.replace(['unknown_0', 'unknown_1'], 'unfilled', inplace = True)

Capstone.electric_vehicle_buy_reason_new_tech_interest.replace(['unknown_0', 'unknown_1'], 'unfilled', inplace = True)

Capstone.electric_vehicle_buy_reason_performance.replace(['unknown_0', 'unknown_1'], 'unfilled', inplace = True)

Capstone.electric_vehicle_buy_reason_quietness.replace(['unknown_0', 'unknown_1'], 'unfilled', inplace = True)

Capstone.electric_vehicle_buy_reason_other_recommended.replace(['unknown_0', 'unknown_1'], 'unfilled', inplace = True)

Capstone.electric_vehicle_buy_again.replace(['unknown_1'], 'unfilled', inplace = True)

# Change all values from str to int
Capstone['residents_under_5'] = pd.to_numeric(Capstone['residents_under_5'])
Capstone['residents_6_to_12'] = pd.to_numeric(Capstone['residents_6_to_12'])
Capstone['residents_13_to_18'] = pd.to_numeric(Capstone['residents_13_to_18'])
Capstone['residents_19_to_24'] = pd.to_numeric(Capstone['residents_19_to_24'])
Capstone['residents_25_to_34'] = pd.to_numeric(Capstone['residents_25_to_34'])
Capstone['residents_35_to_49'] = pd.to_numeric(Capstone['residents_35_to_49'])
Capstone['residents_50_to_64'] = pd.to_numeric(Capstone['residents_50_to_64'])
Capstone['residents_65_and_older'] = pd.to_numeric(Capstone['residents_65_and_older'])

# Create a new column called 'Has_kid'
Capstone['sum_kid'] = [0] * Capstone.shape[0]
Capstone['has_kid'] = [0] * Capstone.shape[0]
Capstone['sum_kid'] = Capstone['residents_under_5'] + Capstone['residents_6_to_12'] + Capstone['residents_13_to_18']
Capstone['has_kid'] = Capstone['sum_kid']
Capstone['has_kid'].replace([1, 2, 3, 4], 'Yes', inplace = True)
Capstone['has_kid'].replace([0], 'No', inplace = True)
Capstone.drop(['sum_kid'], axis = 1, inplace = True)
               
# Create a new column called 'family_size'
Capstone['family_size'] = [0] * Capstone.shape[0]
Capstone['family_size'] = Capstone['residents_under_5'] + Capstone['residents_6_to_12'] + Capstone['residents_13_to_18'] + Capstone['residents_19_to_24'] + Capstone['residents_25_to_34'] + Capstone['residents_35_to_49'] + Capstone['residents_50_to_64'] + Capstone['residents_65_and_older']


Capstone.drop(['residents_under_5', 'residents_6_to_12', 'residents_13_to_18', 'residents_19_to_24', 'residents_25_to_34', 'residents_35_to_49', 'residents_50_to_64', 'residents_65_and_older'], axis = 1, inplace= True)



In [0]:
# for i in range(Capstone.shape[0]-1):
#   print(Capstone[Capstone.columns.tolist()[i]].unique())

In [0]:
Capstone.columns

Index(['dataid', 'vehicle_type', 'model_year', 'quick_charge_port',
       'delivery_date', 'ownership_status', 'localhour', 'car1',
       'any_electric_vehicle', 'status', 'spend_time_at_home_monday',
       'spend_time_at_home_tuesday', 'spend_time_at_home_wednesday',
       'spend_time_at_home_thursday', 'spend_time_at_home_friday',
       'ethnicity_asian_pacific_islander',
       'ethnicity_caucasian_other_than_hispanic_or_latino',
       'ethnicity_hispanic_or_latino', 'ethnicity_decline', 'education_level',
       'total_annual_income', 'electric_vehicle_satisfied',
       'electric_vehicle_features_liked', 'electric_vehicle_buy_reason_envir',
       'electric_vehicle_buy_reason_save_money',
       'electric_vehicle_buy_reason_new_tech_interest',
       'electric_vehicle_buy_reason_performance',
       'electric_vehicle_buy_reason_quietness',
       'electric_vehicle_buy_reason_other_recommended',
       'electric_vehicle_buy_again', 'city', 'state', 'label', 'has_kid',
       

In [0]:
Capstone.head()

dataid vehicle_type model_year quick_charge_port delivery_date  \
0     410       filled     filled             False     unknown_1   
1     410       filled     filled             False     unknown_1   
2     410       filled     filled             False     unknown_1   
3     410       filled     filled             False     unknown_1   
4     410       filled     filled             False     unknown_1   

        ownership_status      localhour       car1 any_electric_vehicle  \
0  Leased by Participant  1/29/17 11:00  unknown_1                  Yes   
1  Leased by Participant  4/20/17 13:00  unknown_1                  Yes   
2  Leased by Participant   5/7/17 12:00  unknown_1                  Yes   
3  Leased by Participant   5/7/17 13:00  unknown_1                  Yes   
4  Leased by Participant  5/10/17 11:00  unknown_1                  Yes   

     status     ...      electric_vehicle_buy_reason_new_tech_interest  \
0  Complete     ...                                           unfilled   
1  Complete     ...                                           unfilled   
2  Complete     ...                                           unfilled   
3  Complete     ...                                           unfilled   
4  Complete     ...                                           unfilled   

   electric_vehicle_buy_reason_performance  \
0   Acceleration and performance of EVâ€™s   
1   Acceleration and performance of EVâ€™s   
2   Acceleration and performance of EVâ€™s   
3   Acceleration and performance of EVâ€™s   
4   Acceleration and performance of EVâ€™s   

   electric_vehicle_buy_reason_quietness  \
0                              Quietness   
1                              Quietness   
2                              Quietness   
3                              Quietness   
4                              Quietness   

   electric_vehicle_buy_reason_other_recommended  electric_vehicle_buy_again  \
0                                       unfilled                          No   
1                                       unfilled                          No   
2                                       unfilled                          No   
3                                       unfilled                          No   
4                                       unfilled                          No   

     city  state  label  has_kid family_size  
0  Austin  Texas      1      Yes           3  
1  Austin  Texas      1      Yes           3  
2  Austin  Texas      1      Yes           3  
3  Austin  Texas      1      Yes           3  
4  Austin  Texas      1      Yes           3  

[5 rows x 35 columns]

## Export the chech_final.csv dataset

In [0]:
from google.colab import drive
drive.mount('drive')

Capstone.to_csv('check_final.csv')
!cp check_final.csv drive/My\ Drive/

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at drive
